In [ ]:
import datetime, os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras.metrics import Accuracy
import tensorflow as tf


In [ ]:
dftemp = pd.read_csv('/kaggle/input/paddy-disease-classification/train.csv')
dftemp.head()

In [ ]:
value_counts = dftemp['label'].value_counts().plot(kind = 'bar')
plt.ylabel("Count")

In [ ]:
dftemp['variety'].value_counts()

In [ ]:
dftemp['age'].value_counts()

In [ ]:
train_ds.class_indices.keys()

In [ ]:
SEED = 548
BATCH_SIZE = 64
EPOCHS = 15
LEARNING_RATE = 0.13
#IMAGE_SIZE = 112
IMAGE_SIZE = 256

img_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)

In [ ]:
train_ds = img_gen.flow_from_directory('../input/paddy-disease-classification/train_images',
                                       target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                       shuffle=True, 
                                       seed=SEED,
                                       class_mode='categorical',
                                       batch_size=BATCH_SIZE,
                                       subset="training")

classes = list(train_ds.class_indices.keys())

print('Classes: '+str(classes))

num_classes  = len(classes)

In [ ]:
val_ds = img_gen.flow_from_directory('../input/paddy-disease-classification/train_images',
                                       target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                       shuffle=False, 
                                       seed=SEED,
                                       class_mode='categorical',
                                       batch_size=BATCH_SIZE,
                                       subset="validation")

In [ ]:
test_ds = img_gen.flow_from_directory('../input/paddy-disease-classification/test_images',
                                      target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                      classes=['.'],
                                      class_mode= None,
                                      batch_size=BATCH_SIZE)

In [ ]:
nb_train_samples = train_ds.samples

steps_per_epoch = nb_train_samples // BATCH_SIZE

In [ ]:
plt.figure(figsize=(30,30))
images = train_ds.next()*255
for i, (image, l) in enumerate(zip(images[0][:9],images[1][:9])):
    ax = plt.subplot(330 + 1 + i)
    ax.set_title( classes[np.argmax(l)] )
    plt.imshow(image)
plt.show()

# model 3

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(8192, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Adding dropout for regularization
    
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Adding dropout for regularization
    
    tf.keras.layers.Dense(128, activation='relu'),
    
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
model.summary()


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Define callback functions
checkpoint = ModelCheckpoint("model_3.h5", monitor="val_loss", save_best_only=True)
early_stopping = EarlyStopping(monitor="val_loss", patience=5)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=0.0001)

# Create a list of callbacks to be passed to the fit method
callbacks = [checkpoint, early_stopping, reduce_lr]

In [ ]:
history = model.fit(
    train_ds,
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=val_ds,
    callbacks=callbacks
)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Plot accuracy
ax1.plot(history.history["accuracy"], color="r")
ax1.plot(history.history["val_accuracy"], color="g")
ax1.legend(["Training", "Validation"])
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Accuracy")
ax1.grid()
ax1.set_ylim(0, 1)

# Plot loss
ax2.plot(history.history["loss"], color="r")
ax2.plot(history.history["val_loss"], color="g")
ax2.legend(["Training", "Validation"])
ax2.set_xlabel("Epochs")
ax2.set_ylabel("Loss")
ax2.grid()

# Show the subplots
plt.tight_layout()
plt.show()

In [ ]:
model.save("my_model_3_after.h5")

In [ ]:
!pip install tf-explain

In [ ]:
import numpy as np
from tf_explain.core.grad_cam import GradCAM

# Load your trained model from the H5 file
model = keras.models.load_model('/kaggle/working/my_model_3.h5')

# Define a function to generate Grad-CAM visualizations
def generate_grad_cam_images(model, images, layer_name, num_images=5):
    for i in range(num_images):
        image = images[0][i]
        label = classes[np.argmax(images[1][i])]
        explanation = GradCAM()
        
        # Get the predicted class index for the image
        predicted_class_index = np.argmax(model.predict(np.expand_dims(image, axis=0)))
        
        # Generate Grad-CAM heatmap
        explanation = GradCAM()
        heat_map = explanation.explain((np.expand_dims(image, axis=0), None), model, class_index=predicted_class_index, layer_name=layer_name)
        
        # Display the original image, Grad-CAM heatmap, and overlay
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 3, 1)
        plt.title("Original Image ({})".format(label))
        plt.imshow(image / 255.0)
        plt.axis("off")

        plt.subplot(1, 3, 2)
        plt.title("Grad-CAM Heatmap")
        plt.imshow(heat_map, cmap="jet")
        plt.axis("off")

        plt.subplot(1, 3, 3)
        plt.title("Overlay")
        plt.imshow(image / 255.0)
        plt.imshow(heat_map, cmap="jet", alpha=0.5)
        plt.axis("off")

        plt.savefig("grad_cam_result_{}.png".format(i))
        plt.show()

# Specify the layer name for which you want to generate Grad-CAM
# You can choose any intermediate layer in your model
layer_name = "conv2d_4"  # Example layer name, change it as needed

# Generate Grad-CAM visualizations for the first 5 images
generate_grad_cam_images(model, images, layer_name, num_images=5)


In [ ]:
import numpy as np
from tf_explain.core.grad_cam import GradCAM

# Load your trained model from the H5 file
model = keras.models.load_model('/kaggle/working/my_model_3.h5')

# Define a function to generate Grad-CAM visualizations
def generate_grad_cam_images(model, images, layer_name, num_images=5):
    for i in range(num_images):
        image = images[0][i]
        label = classes[np.argmax(images[1][i])]
        
        # Get the predicted class index for the image
        predicted_class_index = np.argmax(model.predict(np.expand_dims(image, axis=0)))
        
        # Generate Grad-CAM heatmap
        explanation = GradCAM()
        heat_map = explanation.explain((np.expand_dims(image, axis=0), None), model, class_index=predicted_class_index, layer_name=layer_name)
        
        # Display the original image, Grad-CAM heatmap, and overlay
        plt.figure(figsize=(10, 5))
        
        # Original Image
        plt.subplot(1, 3, 1)
        plt.title("Original Image ({})".format(label))
        plt.imshow(image)
        plt.axis("off")

        # Grad-CAM Heatmap
        plt.subplot(1, 3, 2)
        plt.title("Grad-CAM Heatmap")
        plt.imshow(heat_map, cmap="jet")
        plt.axis("off")

        # Overlay
        plt.subplot(1, 3, 3)
        plt.title("Overlay")
        overlay = image / 255.0 + heat_map
        plt.imshow(overlay)
        plt.axis("off")

        plt.savefig("grad_cam_result_{}.png".format(i))
        plt.show()

# Specify the layer name for which you want to generate Grad-CAM
# You can choose any intermediate layer in your model
layer_name = "conv2d_4"  # Example layer name, change it as needed

# Generate Grad-CAM visualizations for the first 5 images
generate_grad_cam_images(model, images, layer_name, num_images=5)


In [ ]:
import numpy as np
from tf_explain.core.grad_cam import GradCAM

# Load your trained model from the H5 file
model = keras.models.load_model('/kaggle/working/my_model_3.h5')

# Define a function to generate Grad-CAM visualizations
def generate_grad_cam_images(model, images, layer_name, num_images=5):
    for i in range(num_images):
        image = images[0][i]
        label = classes[np.argmax(images[1][i])]
        
        # Get the predicted class index for the image
        predicted_class_index = np.argmax(model.predict(np.expand_dims(image, axis=0)))
        
        # Generate Grad-CAM heatmap
        explanation = GradCAM()
        heat_map = explanation.explain((np.expand_dims(image, axis=0), None), model, class_index=predicted_class_index, layer_name=layer_name)
        
        # Display the original image, Grad-CAM heatmap, and overlay
        plt.figure(figsize=(10, 5))
        
        # Original Image
        plt.subplot(1, 3, 1)
        plt.title("Original Image ({})".format(label))
        plt.imshow(image)
        plt.axis("off")

        # Grad-CAM Heatmap
        plt.subplot(1, 3, 2)
        plt.title("Grad-CAM Heatmap")
        plt.imshow(heat_map, cmap="jet")
        plt.axis("off")

        # Overlay
        plt.subplot(1, 3, 3)
        plt.title("Grad-CAM Heatmap")
        plt.imshow(image)  # Display the original image
        plt.imshow(heat_map, cmap="jet", alpha=0.5)  # Overlay the heatmap with transparency
        plt.axis("off")

        plt.savefig("grad_cam_result_{}.png".format(i))
        plt.show()

# Specify the layer name for which you want to generate Grad-CAM
# You can choose any intermediate layer in your model
layer_name = "conv2d_4"  # Example layer name, change it as needed

# Generate Grad-CAM visualizations for the first 5 images
generate_grad_cam_images(model, images, layer_name, num_images=5)


In [ ]:
import numpy as np
from tf_explain.core.grad_cam import GradCAM

# Load your trained model from the H5 file
model = keras.models.load_model('/kaggle/working/my_model_3.h5')

# Define a function to generate Grad-CAM visualizations
def generate_grad_cam_images(model, images, layer_name, num_images=5):
    for i in range(num_images):
        image = images[0][i]
        label = classes[np.argmax(images[1][i])]
        
        # Get the predicted class index for the image
        predicted_class_index = np.argmax(model.predict(np.expand_dims(image, axis=0)))
        
        # Generate Grad-CAM heatmap
        explanation = GradCAM()
        heat_map = explanation.explain((np.expand_dims(image, axis=0), None), model, class_index=predicted_class_index, layer_name=layer_name)
        
        # Create a colorbar image
        plt.figure(figsize=(1, 5))
        plt.imshow(np.arange(0, 1, 0.01).reshape(1, -1), cmap="jet")
        plt.gca().set_visible(False)
        cax = plt.axes([0.1, 0.2, 0.03, 0.6])
        plt.colorbar(cax=cax)
        plt.title("Heatmap Legend")
        
        # Display the original image, Grad-CAM heatmap, overlay, and colorbar
        plt.figure(figsize=(15, 5))
        
        # Original Image
        plt.subplot(1, 4, 1)
        plt.title("Original Image ({})".format(label))
        plt.imshow(image)
        plt.axis("off")

        # Grad-CAM Heatmap
        plt.subplot(1, 4, 2)
        plt.title("Grad-CAM Heatmap")
        plt.imshow(heat_map, cmap="jet")
        plt.axis("off")

        # Overlay
        plt.subplot(1, 4, 3)
        plt.title("Overlay")
        plt.imshow(image)  # Display the original image
        plt.imshow(heat_map, cmap="jet", alpha=0.5)  # Overlay the heatmap with transparency
        plt.axis("off")
        
        # Colorbar Legend
        plt.subplot(1, 4, 4)
        plt.axis('off')

        plt.savefig("grad_cam_result_{}.png".format(i))
        plt.show()

# Specify the layer name for which you want to generate Grad-CAM
# You can choose any intermediate layer in your model
layer_name = "conv2d_4"  # Example layer name, change it as needed

# Generate Grad-CAM visualizations for the first 5 images
generate_grad_cam_images(model, images, layer_name, num_images=5)


In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

# Evaluate the model on the validation dataset
val_results = model.evaluate(val_ds)

# Predict labels for the validation dataset
val_pred_probs = model.predict(val_ds)
val_pred_labels = np.argmax(val_pred_probs, axis=1)

# Extract true labels from val_ds
val_true_labels = val_ds.classes

# Compute precision, recall, F1 score, and accuracy
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

precision = precision_score(val_true_labels, val_pred_labels, average='weighted')
recall = recall_score(val_true_labels, val_pred_labels, average='weighted')
f1 = f1_score(val_true_labels, val_pred_labels, average='weighted')
accuracy = accuracy_score(val_true_labels, val_pred_labels)

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Accuracy: {accuracy:.4f}')

# Create a confusion matrix
conf_matrix = confusion_matrix(val_true_labels, val_pred_labels)

# Visualize the confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(num_classes)
plt.xticks(tick_marks, classes, rotation=90)
plt.yticks(tick_marks, classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


In [ ]:
# Create a confusion matrix
conf_matrix = confusion_matrix(val_true_labels, val_pred_labels)
import seaborn as sns
# Visualize the confusion matrix with values and angled x-axis labels
plt.figure(figsize=(6, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes, cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.xticks(rotation=90)
plt.show()

In [ ]:
pip install visualkeras

In [ ]:
import visualkeras
from visualkeras import layered_view
layered_model = layered_view(model, legend=True)
# Save the visualization to an image file
layered_model.save('model_visualization.png')

# Display the visualization (optional)
from IPython.display import Image
Image(filename='model_visualization.png')

In [ ]:
history = model.fit(
    train_ds,
    steps_per_epoch = steps_per_epoch,
    epochs = 15,
    validation_data = val_ds
)

#model.summary()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Plot accuracy
ax1.plot(history.history["accuracy"], color="r")
ax1.plot(history.history["val_accuracy"], color="g")
ax1.legend(["Training", "Validation"])
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Accuracy")
ax1.grid()
ax1.set_ylim(0, 1)

# Plot loss
ax2.plot(history.history["loss"], color="r")
ax2.plot(history.history["val_loss"], color="g")
ax2.legend(["Training", "Validation"])
ax2.set_xlabel("Epochs")
ax2.set_ylabel("Loss")
ax2.grid()

# Show the subplots
plt.tight_layout()
plt.show()

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

# Evaluate the model on the validation dataset
val_results = model.evaluate(val_ds)

# Predict labels for the validation dataset
val_pred_probs = model.predict(val_ds)
val_pred_labels = np.argmax(val_pred_probs, axis=1)

# Extract true labels from val_ds
val_true_labels = val_ds.classes

# Compute precision, recall, F1 score, and accuracy
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

precision = precision_score(val_true_labels, val_pred_labels, average='weighted')
recall = recall_score(val_true_labels, val_pred_labels, average='weighted')
f1 = f1_score(val_true_labels, val_pred_labels, average='weighted')
accuracy = accuracy_score(val_true_labels, val_pred_labels)

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Accuracy: {accuracy:.4f}')

# Create a confusion matrix
conf_matrix = confusion_matrix(val_true_labels, val_pred_labels)

# Visualize the confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(num_classes)
plt.xticks(tick_marks, classes, rotation=90)
plt.yticks(tick_marks, classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


In [ ]:
# Create a confusion matrix
conf_matrix = confusion_matrix(val_true_labels, val_pred_labels)
import seaborn as sns
# Visualize the confusion matrix with values and angled x-axis labels
plt.figure(figsize=(6, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes, cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.xticks(rotation=90)
plt.show()

# model 2

In [ ]:



# Create the model
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(8192, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Add dropout to reduce overfitting
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Add dropout to reduce overfitting
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.summary()

model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:

model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_ds,
    steps_per_epoch = steps_per_epoch,
    epochs = 15,
    validation_data = val_ds
)

#model.summary()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Plot accuracy
ax1.plot(history.history["accuracy"], color="r")
ax1.plot(history.history["val_accuracy"], color="g")
ax1.legend(["Training", "Validation"])
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Accuracy")
ax1.grid()
ax1.set_ylim(0, 1)

# Plot loss
ax2.plot(history.history["loss"], color="r")
ax2.plot(history.history["val_loss"], color="g")
ax2.legend(["Training", "Validation"])
ax2.set_xlabel("Epochs")
ax2.set_ylabel("Loss")
ax2.grid()

# Show the subplots
plt.tight_layout()
plt.show()

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

# Evaluate the model on the validation dataset
val_results = model.evaluate(val_ds)

# Predict labels for the validation dataset
val_pred_probs = model.predict(val_ds)
val_pred_labels = np.argmax(val_pred_probs, axis=1)

# Extract true labels from val_ds
val_true_labels = val_ds.classes

# Compute precision, recall, F1 score, and accuracy
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

precision = precision_score(val_true_labels, val_pred_labels, average='weighted')
recall = recall_score(val_true_labels, val_pred_labels, average='weighted')
f1 = f1_score(val_true_labels, val_pred_labels, average='weighted')
accuracy = accuracy_score(val_true_labels, val_pred_labels)

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Accuracy: {accuracy:.4f}')

# Create a confusion matrix
conf_matrix = confusion_matrix(val_true_labels, val_pred_labels)

# Visualize the confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(num_classes)
plt.xticks(tick_marks, classes, rotation=90)
plt.yticks(tick_marks, classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


In [ ]:
import visualkeras
from visualkeras import layered_view
layered_model = layered_view(model, legend=True)
# Save the visualization to an image file
layered_model.save('model_visualization2.png')

# Display the visualization (optional)
from IPython.display import Image
Image(filename='model_visualization2.png')

In [ ]:
# Data augmentation
img_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

In [ ]:
print(IMAGE_SIZE)

# model 1

In [ ]:

model = tf.keras.Sequential([
tf.keras.layers.InputLayer(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2, 2),
tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(8192, activation='relu'),
tf.keras.layers.Dense(1024, activation='relu'),
tf.keras.layers.Dense(128, activation='relu'),
tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Early stopping callback
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Learning rate schedule
lr_schedule = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)


In [ ]:
# Training the model
history = model.fit(
    train_ds,
    steps_per_epoch=steps_per_epoch,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=[early_stopping, lr_schedule],
)


In [ ]:
history = model.fit(
    train_ds,
    steps_per_epoch=steps_per_epoch,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=[early_stopping_callback]
)
